In [ ]:
!nvidia-smi

# EYE GAZE SHARED TASK

In [ ]:
import numpy as np
import pandas as pd
import re
import string
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
from sklearn.metrics import r2_score

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

### DOWNLOAD DATA

In [ ]:
train_data_file_path = "/content/drive/My Drive/CMCL Shared Task/training_data.csv"
#file_path = "/content/drive/MyDrive/datasets/EyeGazeSharedTask/trial_data.csv"

train_data = pd.read_csv(train_data_file_path)

In [ ]:
print("Train Data Shape = ", train_data.shape)
train_data

### BASIC MODEL PLAN

In [ ]:
#@title
import numpy as np
file_path = "/content/drive/My Drive/CMCL Shared Task/glove.6B.200d.txt"

def loadGloveModel(File):
    print("Loading Glove Model")
    f = open(File,'r')
    gloveModel = {}
    for line in f:
        splitLines = line.split()
        word = splitLines[0]
        wordEmbedding = np.array([float(value) for value in splitLines[1:]])
        gloveModel[word] = wordEmbedding
    print(len(gloveModel)," words loaded!")
    return gloveModel

glove_embedding_dict = loadGloveModel(file_path)


























































### PREPROCESS DATA

#### REMOVE EOS TOKEN

In [ ]:
#@title
# FUNCTION TO REMOVE THE <EOS> TOKEN IN THE DATASET

def remove_eos(df):
  cnt = 1
  for i in range(df.shape[0]-1):
    if (df.loc[i+1, "sentence_id"] == cnt):
      df.loc[i, "word"] = df.loc[i, "word"][:-5]   # Remove <EOS> for the last word of each sentence.
      cnt += 1 
  s = df.loc[df.shape[0] - 1, "word"]              # Remove <EOS> for last element separately
  df.loc[df.shape[0] - 1, "word"] = s[:-6] 
  return df

### NO OF CHARS FOR EACH WORD

In [ ]:
#@title
# FUNCTION TO CALCULATE THE NUMBER OF CHARACTERS PER WORD
# ADDS THE DATA IN A NEW COLUMN

def char_per_word(df):
  n_chars = []
  for word in df.word:
    n_chars.append(len(str(word)))
  df["n_chars"] = n_chars
  return df

### NO OF CHARS OF WORD - LEMMATIZED WORD

In [ ]:
#@title
# FUNCTION TO CALCULATE THE DIFFERENCE BETWEEN NUMBER OF CHARACTERS IN WORD AND LEMMATIZED WORD
# ADDS AS A NEW COLUMN

from nltk import WordNetLemmatizer

Lemmatizer = WordNetLemmatizer()

def char_per_lemmatized_word(df):
  n_chars = []
  for word in df.word:
    n_chars.append(len(str(word)) - len(Lemmatizer.lemmatize(word)))
  df["n_char_lemmatized"] = n_chars
  return df

### STOP WORD OR NOT

In [ ]:
#@title
import nltk
nltk.download("stopwords")
nltk.download("wordnet")

stopwords = nltk.corpus.stopwords
stop_words = stopwords.words("english")

In [ ]:
#@title
# FUNCTION TO ASSERT WHETHER A WORD IS STOPWORD  OR NOT
# ADDS THE DATA IN A NEW COLUMN

def add_stopword_check(df):
  if_stopword = []
  for word in df.word:
    if_stopword.append(int(word in stop_words))

  df["stopword"] = if_stopword
  return df

### NUMBER OR NOT

In [ ]:
#@title
# FUNCTION TO DEFINE WHETHER IT IS A NUMBER OR NOT
# ADDS THE DATA AS A NEW COLUMN

def add_number_check(df):
  if_number = []
  for word in df.word:
    if_number.append(int(word.isdigit()))

  df["number"] = if_number
  return df

### TF IDF CALCULATION

In [ ]:
#@title
# FUNCTION TO CALCULATE THE TFIDF OF THE TRAINING DATASET
# ADDS THE DATA IN A NEW COLUMN
# ALSO RETURNS A LIST OF THE SENTENCES

import re
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words=None)
bad_words = []
punc = string.punctuation

def remove_punc(word):
  table = str.maketrans('', '', punc)
  return word.translate(table)

def calc_tfidf(df):
  n = np.array(df["sentence_id"])[-1]
  sentences = []
  sentence_tokens = []
  tf_idfs = []
  MAX_LEN = 0
  for i in range(n+1):   
    temp_df = df[df.sentence_id == i]
    sentence = (' ').join(temp_df.word)
    MAX_LEN = max(MAX_LEN, len(sentence))
    sentences.append(sentence)
    sentence_tokens.append([np.array(temp_df.word)])
  tf_idf = vectorizer.fit_transform(sentences)
  for i, word in enumerate(df.word):
    try:
      tf_idfs.append(tf_idf.toarray()[df["sentence_id"][i]][vectorizer.get_feature_names().index(remove_punc(word.lower()))])
    except:
      bad_words.append(word)
      if word in ["a", "A"]:
        tf_idfs.append(0.8)
      else:
        tf_idfs.append(0.01)
  df["tf_idf"] = tf_idfs


  return sentence_tokens, df, bad_words, MAX_LEN

In [ ]:
#@title
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize 
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(sparse = False)

def pos_tag_func(df):
  tags = []
  for word in df.word:
    if word not in string.punctuation:
      tag = nltk.pos_tag(word)[0][1]
    else:
      tag = "PUNC"
    tags.append(tag)
  df["tags"] = tags
  tag_transform = pd.DataFrame(enc.fit_transform(np.array(df.tags).reshape(-1, 1)))
  df = pd.concat((df, tag_transform), axis = 1)
  return df

In [ ]:
#@title
def use_transformed_GPT(df):
  df["GPT"] = df["TRT"] - df["GPT"]
  return df

In [ ]:
# FUNCTION TO PERFORM ALL PREPROCESSING STEPS

def preprocess(df):
  return calc_tfidf(use_transformed_GPT(pos_tag_func(add_number_check(add_stopword_check(char_per_lemmatized_word(char_per_word(remove_eos(df))))))))

sentence_tokens, df, bad_words, MAX_LEN = preprocess(train_data)

df1 = df.copy()
df1

In [ ]:
print("No of words unaccounted for = ", len(bad_words))

In [ ]:
# FUNCTION TO NORMALISE ALL THE TARGET VALUES

from sklearn.preprocessing import StandardScaler

std_scaler = StandardScaler()

def standardize_target(df):
  keys = df.keys()[3:8]
  new_df = pd.DataFrame(std_scaler.fit_transform(df.iloc[:,3:8]), columns = keys)
  df.update(new_df)
  return 
standardize_target(df)

In [ ]:
df

### FORM TARGETS

In [ ]:
# FUNCTION TO FORM TARGETS AND OTHER FEATURES IN SEPARATE DATA STRUCTURES

def form_targets(sentence_tokens, df, MAX_LEN):
  n = np.array(df["sentence_id"])[-1]
  targets = []
  tags = []
  features = {"n_chars" : [],
              "stopword" : [],
              "number" : [],
              "n_char_lemmatized" : [],
              "tf_idf" : [],
              }

  for i in range(n+1):
    feature = {}
    actual_features = {}
    temp_df = df[df.sentence_id == i]
    
    target = [[0, 0, 0, 0, 0] for i in range(MAX_LEN - len(sentence_tokens[i][0]))]
    actual_targets = [list(x) for x in np.array(temp_df.iloc[:, 3:8])]
    target += actual_targets

    for key in features.keys():
      feature[key] = [0 for j in range(MAX_LEN - len(sentence_tokens[i][0]))]
      actual_features[key] = list(np.array(temp_df.loc[:, key]))
      feature[key] += actual_features[key]

    tag = [[0 for j in range(20)] for k in range(MAX_LEN - len(sentence_tokens[i][0]))]
    actual_tag = [list(x) for x in np.array(temp_df.iloc[:, -20:])]
    tag += actual_tag

    for key in features.keys():
      features[key].append(feature[key]) 
    targets.append(target) 
    tags.append(tag)

  return targets, features, tags

targets, features, tags = form_targets(sentence_tokens, df, MAX_LEN)
print("Target Shape = ", np.array(targets, dtype = "object").shape)

### IMPORT TOKENIZERS AND MODELS

In [ ]:
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split

device = "cuda" if torch.cuda.is_available() else "cpu"
device

### EARLY STOPPING CLASS

In [ ]:
#@title
class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=20, verbose=False, delta=0, path='checkpoint.pt', trace_func=print):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
            path (str): Path for the checkpoint to be saved to.
                            Default: 'checkpoint.pt'
            trace_func (function): trace print function.
                            Default: print            
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func
    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            self.trace_func(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss


### FORMING CLASS OF DATA

In [ ]:
# Class for the data
# Contains 4 keys -> Sentences, targets, input_ids(for BERT), attention_mask(for BERT) 

class EyeGaze_dataset(Dataset):
  def __init__(self, sentence_tokens, targets, features, tags, MAX_LEN):
    self.sentences = sentence_tokens               # List of Sentences
    self.max_len = MAX_LEN
    self.targets = targets                   # List of Padded Targets
    self.features = features
    self.tags = tags
    
  def __len__(self):
    return len(self.sentences)               # No of Examples = 100

  def create_glove_embedding(self, sentence, embedding_dict):
    tokens = []
    seq_len = len(sentence)
    pre_padding = [list(embedding_dict["pad"]) for i in range(max(0, self.max_len - seq_len))]
    tokens += pre_padding
    for word in sentence:
      try:
        tokens.append(list(embedding_dict[word]))
      except:
        tokens.append(list(embedding_dict["unk"]))
    return tokens, pre_padding

  def __getitem__(self, index):
    sentence = list(self.sentences[index][0])        # Get the review at the particular index
    target = self.targets[index]                    # Get the target label at the particular index
    embedding, pre_padding = self.create_glove_embedding(sentence, glove_embedding_dict)
    feature = {}
    for key in features.keys():
      feature[key] = features[key][index]
    tag = tags[index]
    
    # The class simply returns a dictionary of the following
    return_dict = {"sentence" : sentence,
                   "target":torch.tensor(target, dtype = torch.float32),
                   "embedding" : torch.tensor(embedding, dtype = torch.float32),
                   "padding": torch.tensor(pre_padding, dtype = torch.long),
                   "tag":torch.tensor(tag, dtype = torch.float32)}
    for key in features.keys():
      return_dict[key] = torch.tensor(feature[key], dtype = torch.float32)

    return return_dict

In [ ]:
# Forms the DataLoaders with test_size = 0.2
# Each DataLoader has the Class dataset as their elements
def create_data_loader(sentences, targets, features, tags, MAX_LEN, batch_size):
  data = EyeGaze_dataset(sentences, targets, features, tags, MAX_LEN)
  train, val = train_test_split(data, test_size = 0.2, shuffle = False)
  train_data_loader = DataLoader(train, batch_size = batch_size, shuffle = False)
  val_data_loader = DataLoader(val, batch_size = batch_size, shuffle = False)
  
  return train_data_loader, val_data_loader

BATCH_SIZE = 1
train_data_loader, val_data_loader = create_data_loader(sentence_tokens, targets, features, tags, MAX_LEN, BATCH_SIZE)

In [ ]:
temp = next(iter(train_data_loader))
print(temp.keys())
keys = ['target', 'embedding', "padding", 'n_chars', 'n_char_lemmatized', 'stopword', 'number', 'tf_idf', "tag"]
for key in keys:
  print(temp[key].size())
print(temp['sentence'])

Sentences Shape = _(100, variable)_

Tokens Shape = _(100, 320)_

Hidden Shape = _(100, 320, 768)_

Final Output Shape = _(100, 320, 5)_

Target Shape = _(100, 320, 5)_

In [ ]:
class FeatureSideModel(torch.nn.Module):
  def __init__(self):
    super(FeatureSideModel, self).__init__()
    self.encoder = torch.nn.LSTM(input_size = 25, hidden_size = 256, num_layers = 2, batch_first = True, dropout = 0.1, bidirectional = True)
    
  def forward(self, features):
    output, h = self.encoder(features)
     
    return output


In [ ]:
class EyeGazeClassifier(torch.nn.Module):
  def __init__(self):
    super(EyeGazeClassifier, self).__init__()
    self.feature_model = FeatureSideModel()
    self.encoder = torch.nn.LSTM(input_size = 200, hidden_size = 256, num_layers = 2, batch_first = True, dropout = 0.1, bidirectional = True)
    self.dense1 = torch.nn.Linear(in_features = 512, out_features = 128)
    self.act = torch.nn.ReLU()
    self.drop = torch.nn.Dropout(p = 0.1)
    self.dense2 = torch.nn.Linear(in_features = 128, out_features = 5)

  def forward(self, tokens, attention_mask, features):
    feature_output = self.feature_model(features)
    output, h = self.encoder(tokens)
    output = (output+feature_output)/2
    output = self.drop(self.act(self.dense1(output)))
    output = self.dense2(output)
    output = output * attention_mask

    return output


In [ ]:
#del model

In [ ]:
model = EyeGazeClassifier()
model = model.to(device)

### TRAINING

In [ ]:
#@title
EPOCHS = 150
#from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = torch.optim.Adam(model.parameters(), lr=8e-6, betas = (0.9, 0.999))
total_steps = len(train_data_loader) * EPOCHS

#scheduler = get_linear_schedule_with_warmup(
 # optimizer,
 # num_warmup_steps=6,
 # num_training_steps=total_steps
#)

loss_fn = nn.L1Loss(reduction = "sum").to(device)  # reduction = "mean" can be used

In [ ]:
#@title
def target_initialise():
  targets = {}
  outputs = {}

  targets["r20"] = torch.tensor([], dtype = torch.float32)
  targets["r21"] = torch.tensor([], dtype = torch.float32)
  targets["r22"] = torch.tensor([], dtype = torch.float32)
  targets["r23"] = torch.tensor([], dtype = torch.float32)
  targets["r24"] = torch.tensor([], dtype = torch.float32)

  outputs["pred_r20"] = torch.tensor([], dtype = torch.float32)
  outputs["pred_r21"] = torch.tensor([], dtype = torch.float32)
  outputs["pred_r22"] = torch.tensor([], dtype = torch.float32)
  outputs["pred_r23"] = torch.tensor([], dtype = torch.float32)
  outputs["pred_r24"] = torch.tensor([], dtype = torch.float32)
  
  return targets, outputs

In [ ]:
#@title
def store_targets(targets, target):
  targets["r20"] = torch.cat([targets["r20"], target[0,:,0].detach().cpu()], dim = 0)
  targets["r21"] = torch.cat([targets["r21"], target[0,:,1].detach().cpu()], dim = 0)
  targets["r22"] = torch.cat([targets["r22"], target[0,:,2].detach().cpu()], dim = 0)
  targets["r23"] = torch.cat([targets["r23"], target[0,:,3].detach().cpu()], dim = 0)
  targets["r24"] = torch.cat([targets["r24"], target[0,:,4].detach().cpu()], dim = 0)

  return targets  

def store_outputs(outputs, output):
  outputs["pred_r20"] = torch.cat([outputs["pred_r20"], output[0,:,0].detach().cpu()], dim = 0)
  outputs["pred_r21"] = torch.cat([outputs["pred_r21"], output[0,:,1].detach().cpu()], dim = 0)
  outputs["pred_r22"] = torch.cat([outputs["pred_r22"], output[0,:,2].detach().cpu()], dim = 0)
  outputs["pred_r23"] = torch.cat([outputs["pred_r23"], output[0,:,3].detach().cpu()], dim = 0)
  outputs["pred_r24"] = torch.cat([outputs["pred_r24"], output[0,:,4].detach().cpu()], dim = 0)
  
  return outputs

In [ ]:
#@title
def train_epoch(model, data_loader, loss_fn, optimizer, device):
  model = model.train()

  losses = []
  targets, outputs = target_initialise()  
  
  r2 = {}
  
  for d in data_loader:
    tokens = d["embedding"].to(device)
    target = d["target"].to(device)
    char_len = torch.unsqueeze(d["n_chars"], dim = 2)
    char_len_lemmatized = torch.unsqueeze(d["n_char_lemmatized"], dim = 2)
    if_stopword = torch.unsqueeze(d["stopword"], dim = 2)
    if_num = torch.unsqueeze(d['number'], dim = 2)
    tfidf = torch.unsqueeze(d["tf_idf"], dim = 2)
    tag = d["tag"]

    targets = store_targets(targets, target)
    extra_feature = torch.cat((char_len, char_len_lemmatized, if_stopword, if_num, tfidf, tag), dim = 2).to(device)
    
    attention_mask = [0 for i in range(d["padding"].size()[1])] + [1 for i in range(MAX_LEN - d["padding"].size()[1])]
    attention_mask = torch.unsqueeze(torch.unsqueeze(torch.tensor(attention_mask), dim = 0), dim = 2).to(device)
    
    output = model(
        tokens = tokens, 
        attention_mask = attention_mask,
        features = extra_feature
      )

    outputs = store_outputs(outputs, output)
    
    loss = loss_fn(output, target)
    losses.append(loss.item())

    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()
    #scheduler.step()
    optimizer.zero_grad()
  
  r2["nFix"] = r2_score(targets["r20"].numpy(), outputs["pred_r20"].numpy())
  r2["FFD"] = r2_score(targets["r21"].numpy(), outputs["pred_r21"].numpy())
  r2["GPT"] = r2_score(targets["r22"].numpy(), outputs["pred_r22"].numpy())
  r2["TRT"] = r2_score(targets["r23"].numpy(), outputs["pred_r23"].numpy())
  r2["fixProp"] = r2_score(targets["r24"].numpy(), outputs["pred_r24"].numpy())
  
  return np.mean(losses), r2    # correct_predictions.double() / n_examples,

In [ ]:

#@title
def eval_model(model, data_loader, loss_fn, device):
  model = model.eval()

  losses = []
  outputs = []
  r2 = {}
  targets, outputs = target_initialise()  
  correct_predictions = 0

  with torch.no_grad():
    for d in data_loader:
      tokens = d["embedding"].to(device)
      target = d["target"].to(device)
      char_len = torch.unsqueeze(d["n_chars"], dim = 2)
      char_len_lemmatized = torch.unsqueeze(d["n_char_lemmatized"], dim = 2)
      if_stopword = torch.unsqueeze(d["stopword"], dim = 2)
      if_num = torch.unsqueeze(d['number'], dim = 2)
      tfidf = torch.unsqueeze(d["tf_idf"], dim = 2)
      tag = d["tag"]

      targets = store_targets(targets, target)
      extra_feature = torch.cat((char_len, char_len_lemmatized, if_stopword, if_num, tfidf, tag), dim = 2).to(device)
      
      attention_mask = [0 for i in range(d["padding"].size()[1])] + [1 for i in range(MAX_LEN - d["padding"].size()[1])]
      attention_mask = torch.unsqueeze(torch.unsqueeze(torch.tensor(attention_mask), dim = 0), dim = 2).to(device)
      
      output = model(
        tokens = tokens, 
        attention_mask = attention_mask,
        features = extra_feature
      )
      
      outputs = store_outputs(outputs, output)
    
      loss = loss_fn(output, target)
      losses.append(loss.item())

  r2["nFix"] = r2_score(targets["r20"].numpy(), outputs["pred_r20"].numpy())
  r2["FFD"] = r2_score(targets["r21"].numpy(), outputs["pred_r21"].numpy())
  r2["GPT"] = r2_score(targets["r22"].numpy(), outputs["pred_r22"].numpy())
  r2["TRT"] = r2_score(targets["r23"].numpy(), outputs["pred_r23"].numpy())
  r2["fixProp"] = r2_score(targets["r24"].numpy(), outputs["pred_r24"].numpy())
  
  return np.mean(losses), r2    # correct_predictions.double() / n_examples,

In [ ]:
#@title
def save(history, best):
  for key, value in history.items():
    best[key] = value[-1]

  return best

def update(history, train_loss, val_loss, train_r2, val_r2):
  
  history['train_loss'].append(train_loss)
  history['val_loss'].append(val_loss)

  history["train_nFix"].append(train_r2["nFix"])
  history["train_FFD"].append(train_r2["FFD"])
  history["train_GPT"].append(train_r2["GPT"])
  history["train_TRT"].append(train_r2["TRT"])
  history["train_fixProp"].append(train_r2["fixProp"])

  history["val_nFix"].append(val_r2["nFix"])
  history["val_FFD"].append(val_r2["FFD"])
  history["val_GPT"].append(val_r2["GPT"])
  history["val_TRT"].append(val_r2["TRT"])
  history["val_fixProp"].append(val_r2["fixProp"])

  return history

In [ ]:
from collections import defaultdict

history = defaultdict(list)
tolerance = 0
best = {}
best = {"val_loss" : 10000}
early_stopping = EarlyStopping(patience = 20, verbose = True)

for epoch in range(EPOCHS):

  print(f'Epoch {epoch + 1}/{EPOCHS}')
  print('-' * 120)

  train_loss, train_r2 = train_epoch(model,
    train_data_loader,    
    loss_fn, 
    optimizer, 
    device)
  

  print(f'Train loss {train_loss} and Train R2 {train_r2}')

  val_loss, val_r2 = eval_model(
    model,
    val_data_loader,
    loss_fn, 
    device
  )

  print(f'Val loss {val_loss} and Val R2 {val_r2}')
  print()

  history = update(history, train_loss, val_loss, train_r2, val_r2)
  
  if val_loss < best["val_loss"]:
    best = save(history, best)
  
  early_stopping(val_loss, model)
  if early_stopping.early_stop:
    print("Stopped Early at at Epoch ", epoch+1)
    break
  model.load_state_dict(torch.load('checkpoint.pt'))


In [ ]:
plt.plot(history["train_loss"], c = "r", label = "Train Loss")
plt.plot(history["val_loss"], c = "g", label = "Validation Loss")
plt.legend()
plt.xlabel("Epochs")
plt.ylabel("L1 Loss")
plt.show()

In [ ]:
plt.plot(history["train_nFix"], c = "r", label = "Train Loss")
plt.plot(history["val_nFix"], c = "g", label = "Validation Loss")
plt.legend()
plt.xlabel("Epochs")
plt.ylabel("nFix R2")
plt.show()

In [ ]:
plt.plot(history["train_FFD"], c = "r", label = "Train Loss")
plt.plot(history["val_FFD"], c = "g", label = "Validation Loss")
plt.legend()
plt.xlabel("Epochs")
plt.ylabel("FFD R2")
plt.show()

In [ ]:
plt.plot(history["train_GPT"], c = "r", label = "Train Loss")
plt.plot(history["val_GPT"], c = "g", label = "Validation Loss")
plt.legend()
plt.xlabel("Epochs")
plt.ylabel("GPT R2")
plt.show()

In [ ]:
plt.plot(history["train_TRT"], c = "r", label = "Train Loss")
plt.plot(history["val_TRT"], c = "g", label = "Validation Loss")
plt.legend()
plt.xlabel("Epochs")
plt.ylabel("TRT R2")
plt.show()

In [ ]:
plt.plot(history["train_fixProp"], c = "r", label = "Train Loss")
plt.plot(history["val_fixProp"], c = "g", label = "Validation Loss")
plt.legend()
plt.xlabel("Epochs")
plt.ylabel("fixProp R2")
plt.show()

In [ ]:
display_data = [[best["train_loss"], best["train_nFix"], best["train_FFD"], best["train_GPT"], best["train_TRT"], best["train_fixProp"]],
 [best["val_loss"], best["val_nFix"], best["val_FFD"], best["val_GPT"], best["val_TRT"], best["val_fixProp"]]]

display_df = pd.DataFrame(display_data, columns = ["L1Loss", "nFix", "FFD", "GPT", "TRT", "fixProp"], index = ["Train", "Val"]) 

In [ ]:
display_df

In [ ]:
display_data2 = [[history["train_loss"][-1], history["train_nFix"][-1], history["train_FFD"][-1], history["train_GPT"][-1], history["train_TRT"][-1], history["train_fixProp"][-1]], 
                 [history["val_loss"][-1], history["val_nFix"][-1], history["val_FFD"][-1], history["val_GPT"][-1], history["val_TRT"][-1], history["val_fixProp"][-1]]]
display_df2 = pd.DataFrame(display_data2, columns = ["L1Loss", "nFix", "FFD", "GPT", "TRT", "fixProp"], index = ["Train ", "Val "]) 

In [ ]:
display_df2

In [ ]:
history["train_loss"][0], history["val_loss"][0]

In [ ]:
save_file_path = "/content/drive/My Drive/CMCL Shared Task/RobertaTokenFeaturesAttentionPosTag.pth"
torch.save(model.state_dict(), save_file_path)

In [ ]:
del model

In [ ]:
'''
IDEAS:
Training :-
1. Sentence Formation.
2. BERT tokenize.
3. Base BERT Model -> if encoded value == 0 -> 5 output Dense Layer.
4. MAE metric for loss calculation.

Test :-
1. Sentence Formation.
2. BERT Tokenize.
3. Base BERT Model  -> if encoded value == 0 -> 5 output Dense Layer.
4. Order is maintained and predictions are pasted on the csv file.
'''

In [ ]:
'''
trainer = Engine(train_epoch)
train_evaluator = Engine(train_epoch)
validation_evaluator = Engine(val_epoch)

Loss(loss_fn).attach(train_evaluator, "l1")
Loss(loss_fn).attach(validation_evaluator, "l1")

def score_function(engine):
    val_loss = engine.state.metrics['nll']
    return val_loss

handler = EarlyStopping(patience = 10, score_function=score_function, trainer = trainer)
validation_evaluator.add_event_handler(Events.COMPLETED, handler)

def log_training_results(engine):
    train_evaluator.run(train_data_loader)
    metrics = train_evaluator.state.metrics
    pbar.log_message(
        "Training Results - Epoch: {} \nMetrics\n{}"
        .format(engine.state.epoch, pprint.pformat(metrics)))
    
def log_validation_results(engine):
    validation_evaluator.run(val_data_loader)
    metrics = validation_evaluator.state.metrics
    metrics = validation_evaluator.state.metrics
    pbar.log_message(
        "Validation Results - Epoch: {} \nMetrics\n{}"
        .format(engine.state.epoch, pprint.pformat(metrics)))
    pbar.n = pbar.last_print_n = 0

trainer.add_event_handler(Events.EPOCH_COMPLETED, log_validation_results)

checkpointer = ModelCheckpoint('checkpoint', 'textcnn', save_interval=1, n_saved=2, create_dir=True, save_as_state_dict=True)

best_model_save = ModelCheckpoint(
    'best_model', 'textcnn', n_saved=1,
    create_dir=True, save_as_state_dict=True,
    score_function=score_function)
trainer.add_event_handler(Events.EPOCH_COMPLETED, checkpointer, {'textcnn': model})
validation_evaluator.add_event_handler(Events.EPOCH_COMPLETED, best_model_save, {'textcnn': model})


trainer.run(train_data_loader, max_epochs=120)
'''